In [ ]:
!pip install financetoolkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.2/356.2 kB 28.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.


In [ ]:
from financetoolkit import Toolkit

In [ ]:
dataset = Toolkit.ratios.collect_all_ratios(
                growth=True
            )

AttributeError: 'property' object has no attribute 'collect_all_ratios'

In [ ]:
!pip install -qU langchain langchain-community langchain-groq langchain-core langchain-pinecone gpt4all langgraph sentence-transformers gradio\
                langchain-huggingface groq ollama cohere langchain pinecone-client PyPDF2 einops html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 15.0 MB/s eta 0:00:00
 

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import json
import re
from langchain_core.runnables import (
    RunnableParallel,
    RunnableBranch,
    RunnablePassthrough,
)
from langchain_core.messages import HumanMessage, AIMessage
import warnings
import PyPDF2
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
from langchain.document_transformers import Html2TextTransformer
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from langchain_core.tools import Tool
from langchain_core.messages.base import BaseMessage
import operator

warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
pdf = PyPDF2.PdfReader("/content/Asian_Paints_FY24_AnnualReport.pdf")
pdf_text = ""
for page in pdf.pages:
    pdf_text += page.extract_text()

KeyboardInterrupt: 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(pdf_text)
chunks = text_splitter.create_documents(texts)

In [ ]:
import pinecone


# Create the vector store from documents
vectorstore_from_docs = PineconeVectorStore.from_documents(
    chunks,
    index_name="finbuddy",
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# Create the vector store from texts
vectorstore_from_texts = PineconeVectorStore.from_texts(
    texts,
    index_name="finbuddy",
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)



In [ ]:
os.environ["PINECONE_API_KEY"] = "534349d3-964c-4282-a45c-6a5a23ec8764"

In [ ]:
vectorstore_from_docs = PineconeVectorStore(
    index_name="finbuddy",
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "What is Inventories?"

results = vectorstore_from_docs.similarity_search(
    query,
    k=3  # return 3 most relevant docs
)

print(results)

[Document(page_content='packing materials, stores, spares, components, \nconsumables and stock-in-trade are carried \nat the lower of cost and net realizable value. \nHowever, materials and other items held for use \nin production of inventories are not written down \nbelow cost if the finished goods in which they \nwill be incorporated are expected to be sold at or \nabove cost. Net realizable value is the estimated \nselling price in the ordinary course of business less \nestimated cost of completion and estimated costs \nnecessary to make the sale.\n  Cost of inventory is determined on a weighted \naverage basis. Cost of inventory comprises all costs \nof purchase, non-refundable duties and taxes, cost \nof conversion including an appropriate share of \nfixed and variable production overheads and all \nother costs incurred in bringing the inventory to \ntheir present location and condition.  The Group considers factors like estimated \nshelf life, product discontinuances and ageing'

In [ ]:
class Retriever :
    def __init__(self,vectorstore):
        self.vecstore=vectorstore
    vecstore=vectorstore_from_docs
    def sim_search(self,query):
        return   self.vecstore.similarity_search(query,k=10)
retriever= Retriever(vectorstore_from_docs)

In [ ]:
os.environ['GROQ_API_KEY'] = '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'

In [ ]:
class VectorStore(BaseModel):
    (
         "A vectorstore contains information about Financial Annual Report"
        ", Balance statements and Company Financial Information"
    )

    query: str

router_prompt_template = (
    "You are an expert in routing user queries to a VectorStore\n"
    "The VectorStore contains Financial Annual Report.\n"
    'Note that if a query is not annual report related, you must output "not context related", don\'t try to use any tool.\n\n'
    "query: {query}"
)

llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0, )
prompt = ChatPromptTemplate.from_template(router_prompt_template)
question_router = prompt | llm.bind_tools(tools=[VectorStore])

In [ ]:
response = question_router.invoke("What is current asset?")
"tool_calls" in response.additional_kwargs

True

In [ ]:
rag_template_str = (
    "You are a helpful assistant. Answer the query below based on the provided context.\n\n"
    "context: {context}\n\n"
    "query: {query}"
)


rag_prompt = ChatPromptTemplate.from_template(rag_template_str)
rag_chain = rag_prompt | llm | StrOutputParser()

query = "Drawbacks of Aisan Paints?"
context = retriever.sim_search(query)

response = rag_chain.invoke({"query": query, "context": context})

Markdown(response)

The Current ratio is calculated as:

Current ratio = Current assets ÷ Current liabilities

However, I don't have the specific numbers for the company's current assets and current liabilities. If you provide me with those numbers, I can help you calculate the Current ratio.

In [ ]:
fallback_prompt = ChatPromptTemplate.from_template(
    (
        "You are a well renowned Finance Analyst your name is 'FinBuddy'.\n"
        "Do not respond to queries that are not related to Finance.\n"
        "If a query is not related to finance, acknowledge your limitations.\n"
        "Provide concise responses to only finance related queries.\n\n"
        "Current conversations:\n\n{chat_history}\n\n"
        "human: {query}"
    )
)

fallback_chain = (
    {
        "chat_history": lambda x: "\n".join(
            [
                (
                    f"human: {msg.content}"
                    if isinstance(msg, HumanMessage)
                    else f"AI: {msg.content}"
                )
                for msg in x["chat_history"]
            ]
        ),
        "query": itemgetter("query") ,
    }
    | fallback_prompt
    | llm
    | StrOutputParser()
)

fallback_chain.invoke(
    {
        "query": "Hello",
        "chat_history": [],
    }
)

"Hello! I'm FinBuddy, a Finance Analyst. How can I assist you with your finance-related queries today?"